In this example kernel, we try to demonstrate the LP for the Sudoku game. To study the problem 

$$\min_{X} \|X\|_{L^1} $$
subject to equality constraint $AX = B$.

In [1]:
import os
print(os.listdir("input"))

['.DS_Store', 'large1.csv', 'large2.csv', 'small1.csv', 'small2.csv']


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as scs # sparse matrix construction 
import scipy.linalg as scl # linear algebra algorithms
import scipy.optimize as sco # for minimization use
import matplotlib.pylab as plt # for visualization
from collections import defaultdict
import math
def fixed_constraints(N=9):
    rowC = np.zeros(N)
    rowC[0] =1
    rowR = np.zeros(N)
    rowR[0] =1
    row = scl.toeplitz(rowC, rowR)
    ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))
    
    colR = np.kron(np.ones((1,N)), rowC)
    col  = scl.toeplitz(rowC, colR)
    COL  = np.kron(col, np.eye(N))
    
    M = int(np.sqrt(N))
    boxC = np.zeros(M)
    boxC[0]=1
    boxR = np.kron(np.ones((1, M)), boxC) 
    box = scl.toeplitz(boxC, boxR)
    box = np.kron(np.eye(M), box)
    BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))
    
    cell = np.eye(N**2)
    CELL = np.kron(cell, np.ones((1,N)))
    
    return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))
#print(fixed_constraints(N=9))

# For the constraint from clues, we extract the nonzeros from the quiz string.
def clue_constraint(input_quiz, N=9):
    m = np.reshape([int(c) for c in input_quiz], (N,N))
    #print(m)
    r, c = np.where(m.T)
    v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
    table = N * c + r
    table = np.block([[table],[v-1]])
    #print(table)
    # it is faster to use lil_matrix when changing the sparse structure.
    CLUE = scs.lil_matrix((len(table.T), N**3))

    for i in range(len(table.T)):
        CLUE[i,table[0,i]*N + table[1,i]] = 1
    # change back to csr_matrix.
    CLUE = CLUE.tocsr() 
    
    return CLUE
#print(clue_constraint( '000000000000000127009002530000000000004000981080754000000000000060089003473105000', N=9))

In [3]:
def clean(x):
    if x > 9:
        x = 0
    return x

In [4]:
# Initial version: bug exists
# def forward_checking(input_quiz , N=9):
#     m = np.reshape([int(c) for c in input_quiz], (N,N))
#     #r, c = np.where(m.T)
#     #v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
#     for i in range(N):
#         for j in range(N):
#             if m[i][j] == 0:
#                 m[i][j] = 123456789
#                 for k in m[i]:
#                     if k < 10:
#                         s = str(m[i][j]).replace(str(k), '0')
#                         m[i][j] = int(s)
#                 for k in m[:,j]:
#                     if k < 10:
#                         s = str(m[i][j]).replace(str(k), '0')
#                         m[i][j] = int(s)
#                 dd = my_block(m) #block dict
#                 ll = dd[(math.ceil(i/N), math.ceil(j/N))]
#                 #for k in ll:         #eliminate block
#                     #if k < 10:
#                         #s = str(m[i][j]).replace(str(k), '0')
#                         #m[i][j] = int(s)
                
#     return m

# Version 2: correct, no box check
# def forward_checking(input_quiz , N=9):
# #     print(input_quiz)
#     m = np.reshape([int(c) for c in input_quiz],(N,N))
    
#     for i in range(N):
#         for j in range(N):
#            # print(i, j,m[i][j])
#             if m[i][j] == 0:
#                 m[i][j] = 123456789
               
#                 for k in m[i]:
#                    # print(m)
#                    # print("k",k)
#                     if k < 10 and k != 0:
#                         string = str(m[i][j])
#                         if str(k) in string:
#                             if len(string) > 1:
#                                 string = string.replace(str(k), '')
#                        # print("yes",string)
#                         m[i][j] = int(string)
                        
                        
#                 for k in m[:,j]:
#                    # print("k",k)
#                     if k < 10 and k != 0:
#                         string = str(m[i][j])
#                         if str(k) in string and len(string) > 1:
#                             string = string.replace(str(k), '')
#                        # print("no",string)
#                         m[i][j] = int(string)
#     for i in range(N):
#         for j in range(N):
#             if m[i][j] > 9:
#                 m[i][j] = 0
                
#     return m
def my_block(mygb, N=9):
    dd = defaultdict(list)
    for i in range(N):
        for j in range(N):
            key = (math.floor(i/3), math.floor(j/3))
            dd[key].append(mygb[i][j])
    return dd

def forward_checking(input_quiz , N=9):
#     print(input_quiz)
    m = np.reshape([int(c) for c in input_quiz],(N,N))
    
    for i in range(N):
        for j in range(N):
           # print(i, j,m[i][j])
            if m[i][j] == 0:
                m[i][j] = 123456789
               
                for k in m[i]:
                   # print(m)
                   # print("k",k)
                    if k < 10 and k != 0:
                        string = str(m[i][j])
                        if str(k) in string:
                            if len(string) > 1:
                                string = string.replace(str(k), '')
                       # print("yes",string)
                    
                        m[i][j] = int(string)
#                         if i == 5 and j == 6:
#                             print("row,",m[i][j])
                        
                        
                for k in m[:,j]:
                   # print("k",k)
                    if k < 10 and k != 0:
                        string = str(m[i][j])
                        if str(k) in string and len(string) > 1:
                            string = string.replace(str(k), '')
                       # print("no",string)
                       
        
                        m[i][j] = int(string)
#                         if i == 5 and j == 6:
#                             print("col",m[i][j])
                        
                        
                dd = my_block(m) #block dict
#                 print("dd")
#                 print(dd)
#                 print("i,j",i,j)
#                 print("dd",dd)

                
                ll = dd[(math.floor(i/3), math.floor(j/3))]
#                 print("ll")
#                 print(ll)
#                 print("ll")
#                 print(ll)
    
                for k in ll:         #eliminate block
                    if k < 10 and k != 0:
                        string = str(m[i][j])
                        if str(k) in string and len(string) > 1:
                            string = string.replace(str(k), '')
#                         print("hello",string)
                        
                        m[i][j] = int(string)
#                         if i == 5 and j == 6:
#                             print("box",m[i][j])
    for i in range(N):
        for j in range(N):
            if m[i][j] > 9:
                m[i][j] = 0
                
    return m


print(forward_checking( '000000000000000127009002530000000000004000981080754000000000000060089003473105000', N=9))


[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 2 7]
 [0 0 9 0 0 2 5 3 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 9 8 1]
 [0 8 0 7 5 4 0 6 2]
 [0 0 0 0 0 0 0 0 0]
 [0 6 0 0 8 9 0 0 3]
 [4 7 3 1 0 5 0 9 0]]


In [6]:
def forward_tester(sol, result):
    if np.sum(result*(sol!=result))>0:
        print(sol)
        print(result)
        return False
    else:
        return True

In [5]:
small2 = pd.read_csv('input/small2.csv')
small1 = pd.read_csv('input/small1.csv')
large2 = pd.read_csv('input/large2.csv')
large1 = pd.read_csv('input/large1.csv')


In [9]:
test1 = small1['quizzes'][0]
sol1 = small1['solutions'][0]
test2 = small2['quizzes'][0]
sol2 = small2['solutions'][0]

In [9]:
N = 1000
n = 0
for i in range(N):
    sol = large2['solutions'][i]
    test = large2['quizzes'][i]
    sol_mat = np.reshape([int(c) for c in sol], (9,9))
    test_mat = np.reshape([int(c) for c in test], (9,9))
    result = forward_checking(test)
    if forward_tester(sol_mat,result):
        n +=1
print('sucess percentage:', n/N)

sucess percentage: 1.0


In [14]:
N = 10
n = 0
for i in range(N):
    sol = small2['solutions'][i]
    test = small2['quizzes'][i]
    sol_mat = np.reshape([int(c) for c in sol], (9,9))
    test_mat = np.reshape([int(c) for c in test], (9,9))
    result = forward_checking(test)
#     clean_result = np.vectorize(clean)(result)
    if np.any(result!=test_mat):
        print(i)
        print(sol_mat)
        print(test_mat)
        print(result)
        print()
        n +=1
print('sucess percentage:', n/N)

2
[[6 5 3 2 8 7 9 4 1]
 [7 9 4 6 3 1 2 5 8]
 [1 2 8 9 4 5 3 7 6]
 [8 1 9 7 2 4 5 6 3]
 [2 3 6 8 5 9 4 1 7]
 [5 4 7 1 6 3 8 2 9]
 [9 6 5 3 7 2 1 8 4]
 [3 7 2 4 1 8 6 9 5]
 [4 8 1 5 9 6 7 3 2]]
[[0 0 0 0 0 7 0 0 0]
 [0 9 0 0 0 1 0 0 0]
 [0 0 0 0 4 5 0 0 6]
 [0 0 0 0 2 0 0 0 0]
 [0 3 6 0 0 0 4 1 0]
 [5 0 0 0 0 0 8 0 9]
 [0 0 0 0 0 0 0 0 4]
 [0 0 0 0 1 8 0 0 0]
 [0 8 1 5 0 0 0 3 2]]
[[0 0 0 0 0 7 0 0 0]
 [0 9 0 0 0 1 0 0 0]
 [0 0 0 0 4 5 0 0 6]
 [0 0 0 0 2 0 0 0 0]
 [0 3 6 0 0 9 4 1 0]
 [5 0 0 0 0 0 8 0 9]
 [0 0 0 0 0 0 0 0 4]
 [0 0 0 0 1 8 0 0 0]
 [0 8 1 5 0 0 0 3 2]]

3
[[1 5 2 4 7 9 6 8 3]
 [3 6 8 2 1 5 9 7 4]
 [9 7 4 6 3 8 5 1 2]
 [4 1 6 3 8 7 2 5 9]
 [7 8 3 9 5 2 4 6 1]
 [5 2 9 1 4 6 8 3 7]
 [2 3 7 8 6 4 1 9 5]
 [8 9 1 5 2 3 7 4 6]
 [6 4 5 7 9 1 3 2 8]]
[[0 5 2 4 7 0 0 0 0]
 [0 6 0 0 0 0 0 0 0]
 [0 0 0 0 0 8 0 1 0]
 [4 0 0 0 0 0 0 0 9]
 [7 0 0 9 5 0 0 0 0]
 [0 2 0 0 4 0 0 3 0]
 [0 0 0 8 0 0 0 9 0]
 [0 0 0 0 0 3 7 0 6]
 [0 0 0 0 9 1 0 0 0]]
[[0 5 2 4 7 0 0 0 0]
 [0 6 0 0 0 0 0 0 0]
 [0

Use ``scipy.optmize`` to find the solution. We test the method on ``large1`` with random seed 42 to generate the 1000 samples.

In [ ]:
import time

# We test the following algoritm on small data set.
data = pd.read_csv("../input/small2.csv") 

corr_cnt = 0
start = time.time()

random_seed = 42
np.random.seed(random_seed)

if len(data) > 1000:
    samples = np.random.choice(len(data), 1000)
else:
    samples = range(len(data))

for i in range(len(samples)):
    quiz = data["quizzes"][samples[i]]
    solu = data["solutions"][samples[i]]
    A0 = fixed_constraints()
    A1 = clue_constraint(quiz)

    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])


    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S@vh
    B = u.T@B
    B = B[:K]

    c = np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ])
    G = np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]])
    h = np.zeros(A.shape[1]*2)
    H = np.block([A, -A])
    b = B

    ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
    x = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]

    
    z = np.reshape(x, (81, 9))
    if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
                      - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
        pass
    else:
        #print("CORRECT")
        corr_cnt += 1

    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )